In [1]:
!pip install tensorflow==2.1

In [4]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.1.0


In [5]:
users = ['Ryan','Alan']
movies = ['Star Wars', 'The Dark Knight', 'Shrek', 'The Incredibles', 'Bleu', 'Memento']
features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']

In [6]:
num_users = len(users)
num_movies = len(movies)
num_feats = len(features)
num_recommendations = 2


In [7]:
users_movies = tf.constant([
                [2,  5, 0 ,  0, 8, 0],
                [0,  0, 4 , 10, 8, 0]],dtype=tf.float32)

In [8]:
movies_feats = tf.constant([
                [1, 1, 0, 0, 1],
                [1, 1, 0, 0, 0],
                [0, 0, 1, 1, 0],
                [1, 0, 1, 1, 0],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 0, 1]],dtype=tf.float32)


In [9]:

users_feats = tf.matmul(users_movies,movies_feats)
users_feats

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[ 7.,  7.,  0.,  0., 10.],
       [10.,  0., 14., 14.,  8.]], dtype=float32)>

In [10]:
users_feats = users_feats/tf.reduce_sum(users_feats,axis=1,keepdims=True)
users_feats

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[0.29166666, 0.29166666, 0.        , 0.        , 0.41666666],
       [0.2173913 , 0.        , 0.3043478 , 0.3043478 , 0.17391305]],
      dtype=float32)>

In [11]:

top_users_features = tf.nn.top_k(users_feats, num_feats)[1]
top_users_features

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[4, 0, 1, 2, 3],
       [2, 3, 0, 4, 1]])>

In [12]:
for i in range(num_users):
    feature_names = [features[int(index)] for index in top_users_features[i]]
    print('{}: {}'.format(users[i],feature_names))


Ryan: ['Drama', 'Action', 'Sci-Fi', 'Comedy', 'Cartoon']
Alan: ['Comedy', 'Cartoon', 'Action', 'Drama', 'Sci-Fi']


In [13]:
#We will use the dot product as our similarity measure. In essence, this is a weighted movie average for each user.

users_ratings = tf.matmul(users_feats,tf.transpose(movies_feats))
users_ratings

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[1.        , 0.5833333 , 0.        , 0.29166666, 0.41666666,
        0.7083333 ],
       [0.39130434, 0.2173913 , 0.6086956 , 0.82608694, 0.17391305,
        0.39130434]], dtype=float32)>

In [14]:
#If a user has already rated a movie, we ignore that rating. This way, we only focus on ratings for previously unseen/unrated movies.

users_ratings_new = tf.where(tf.equal(users_movies, tf.zeros_like(users_movies)),
                                  users_ratings,
                                  tf.zeros_like(tf.cast(users_movies, tf.float32)))
users_ratings_new

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.29166666, 0.        ,
        0.7083333 ],
       [0.39130434, 0.2173913 , 0.        , 0.        , 0.        ,
        0.39130434]], dtype=float32)>

In [15]:
top_movies = tf.nn.top_k(users_ratings_new, num_recommendations)[1]
top_movies

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[5, 3],
       [0, 5]])>

In [16]:
for i in range(num_users):
    movie_names = [movies[index] for index in top_movies[i]]
    print('{}: {}'.format(users[i],movie_names))

Ryan: ['Memento', 'The Incredibles']
Alan: ['Star Wars', 'Memento']
